<h1> Project: Housing Evictions and Fair Market Rents in New York City</h1> <a id=7></a>
<h3> Mapping Tests for Eviction and FMR data</h3>


### Page Summary:
This page is meant to draft the mapping of both evictions and FMR datasets.

## Data <a id=3></a>

For this project, we are utilizing data from two datasets (evictionlab.org and HUD) that both include zip code geolocators, in order to both map and analyze the data.  In order to make this happen, we needed to utilize a "crosswalk" dataset from HUD.gov in order to connect geographic areas.  Additionally, we will use ACS data for demographic information within each area.

Our first task is to read in and inspect the merged <b>Eviction Lab</b> and <b>FMR</b> dataset.  In the eviction dataset, there are 8428 unique observations and 7 variables, and in the merged dataset, there are 9744 observations across 21 columns.  

In [1]:
import pandas as pd         #quick stats       
import numpy as np      #numerical functions
import matplotlib.pyplot as plt    #visualization library
import seaborn as sns    #visualization and stats

In [2]:
%matplotlib inline

In [3]:
#load Evict_FMR_merged dataset
path = '/Users/ameliaingram/Documents/My_GitHub+Repository/eviction-rent/data/Evict_FMR_merged.csv'

df = pd.read_csv(path, header=0)            # read eviction data from online


In [16]:
df.info()                               # returns # rows/obs, columns/variables and types

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9744 entries, 0 to 9743
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   GEOID                      9744 non-null   int64  
 1   filings_2020               9744 non-null   int64  
 2   filings_avg                9744 non-null   float64
 3   racial_majority            9744 non-null   object 
 4   month                      9744 non-null   object 
 5   ZIP                        9744 non-null   int64  
 6   HUD Area Code_x            9744 non-null   object 
 7   HUD Metro FMR Area Name_x  9744 non-null   object 
 8   SAFMR22 2BR                9744 non-null   int64  
 9   SAFMR22 2BR 90% Pmt        9744 non-null   int64  
 10  SAFMR22 2BR 110% Pmt       9744 non-null   int64  
 11  HUD Area Code_y            9744 non-null   object 
 12  HUD Metro FMR Area Name_y  9744 non-null   object 
 13  SAFMR21 2BR                9744 non-null   int64

There are two main issues we see above. First, we can observe `GEOID` cases are essentially zip codes and a number of `GEOID` cases that are listed as "sealed".  According to the data dictionary from the Eviction Lab website, "A modest portion of filings are reported to us with missing, incorrect, or out-of-bounds addresses. In these cases, we do not assign a Census Tract or Zip code to the case." There were 29 sealed cases excluded for the purposes of merging and mapping.  If we wish to use the sealed sets, we can load the `f_evict` dataset below.<br>

Secondly, the `HUD Metro Area Name` actually changed for certain zip codes from year to year.  We kept those columns in the data for reference.  Not sure if it's important yet, but they're there.

The `month` variable includes both month and year data.  Will need to confirm they are usable in Python's date format.  The 'sealed' values from the `GEOID` column now has 896 null entries.  Hopefully this won't be a problem--it is relatively low.

In [13]:
#load Evictions dataset
path2 = '/Users/ameliaingram/Documents/My_GitHub+Repository/eviction-rent/data/evictions_allcities_monthly_2020_2021.csv'

df2 = pd.read_csv(path2, header=0)            # read eviction data from online
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256865 entries, 0 to 256864
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   city             256865 non-null  object 
 1   type             256865 non-null  object 
 2   GEOID            256865 non-null  object 
 3   racial_majority  255969 non-null  object 
 4   month            256865 non-null  object 
 5   filings_2020     256865 non-null  int64  
 6   filings_avg      256865 non-null  float64
 7   last_updated     256865 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 15.7+ MB


### Check for missing values
Before any initial analysis, we need to check for missing values from each dataset. In the merged dataset, there are no missing values.  However in the Evictions dataset, the controls `GEOID` and `racial_majority` had 896 missing values, and the rest are fine.  In the context of this large dataset these are acceptable missing amounts to continue to use everything. 

In [19]:
df.isnull().sum()        # confirm the number of NaN values for the df

GEOID                        0
filings_2020                 0
filings_avg                  0
racial_majority              0
month                        0
ZIP                          0
HUD Area Code_x              0
HUD Metro FMR Area Name_x    0
SAFMR22 2BR                  0
SAFMR22 2BR 90% Pmt          0
SAFMR22 2BR 110% Pmt         0
HUD Area Code_y              0
HUD Metro FMR Area Name_y    0
SAFMR21 2BR                  0
SAFMR21 2BR 90% Pmt          0
SAFMR21 2BR 110% Pmt         0
CBSASub20                    0
HUD Metro FMR Area Name20    0
SAFMR20 2BR                  0
SAFMR20 2BR 90% Pmt          0
SAFMR20 2BR 110% Pmt         0
dtype: int64

In [20]:
#Convert 'GEOID' sealed entries into NaN
df2['GEOID'] = df2.GEOID.replace('sealed', np.nan)

df2.isnull().sum()        # returns the number of NaN values for the df

city                 0
type                 0
GEOID              896
racial_majority    896
month                0
filings_2020         0
filings_avg          0
last_updated         0
dtype: int64

I will first rename `race_majority` to `race` in order to ease analysis.  Then, I will also add `counties` to the dataset by assigning zip codes to counties and then applying those as a function to the `GEOID` info for readability in the analysis and plots.

In [21]:
#rename of race column in Eviction dataset here
df2 = df2.rename(columns={'race_majority':'Race'})

df2.columns

Index(['city', 'type', 'GEOID', 'racial_majority', 'month', 'filings_2020',
       'filings_avg', 'last_updated'],
      dtype='object')

Now the variables are ready to perform an initial exploratory analysis.  

## 1. Mapping of Eviction Data<a id=3.1></a>



In [24]:
#import packages
import geopandas as gp
import folium as fm
from folium.plugins import HeatMap


In [31]:
#rename 'ZIP' to 'zipcode' for mapping purposes
#rename columns here to enable the second part of the merge
df = df.rename(columns={'ZIP':'zipcode'})

df.columns

Index(['GEOID', 'filings_2020', 'filings_avg', 'racial_majority', 'month',
       'zipcode', 'HUD Area Code_x', 'HUD Metro FMR Area Name_x',
       'SAFMR22 2BR', 'SAFMR22 2BR 90% Pmt', 'SAFMR22 2BR 110% Pmt',
       'HUD Area Code_y', 'HUD Metro FMR Area Name_y', 'SAFMR21 2BR',
       'SAFMR21 2BR 90% Pmt', 'SAFMR21 2BR 110% Pmt', 'CBSASub20',
       'HUD Metro FMR Area Name20', 'SAFMR20 2BR', 'SAFMR20 2BR 90% Pmt',
       'SAFMR20 2BR 110% Pmt'],
      dtype='object')

In [36]:
# Set zipcode type to string (folium)
df['zipcode'] = df['zipcode'].astype('str')

# get the mean value across all data points
zipcode_data = df.groupby('zipcode').aggregate(np.mean)
zipcode_data.reset_index(inplace = True)

In [38]:
# count number of filings_2020 grouped by zipcode
df['count'] = 1
temp = df.groupby('zipcode').sum('filings_2020')
temp.reset_index(inplace = True)
temp = temp[['zipcode', 'count']]
zipcode_data = pd.merge(zipcode_data, temp, on='zipcode')
# drop count from df dataset
df.drop(['count'], axis = 1, inplace = True)

In [42]:
# Get geo data file path
geo_data_file = '/Users/ameliaingram/Documents/My_GitHub+Repository/eviction-rent/data/nyc-zip-code-tabulation-areas-polygons.geojson'
#load GEOJSON
with open(geo_data_file, 'r') as jsonFile:
    geo_data = json.load(jsonFile)

tmp = geo_data


NameError: name 'json' is not defined

In [ ]:
#visualize distribution of evictions
def count_dist(df, location, filings_2020):
    group_counts = pd.DataFrame(df.groupby([location, filings_2020]).size().unstack(1))
    group_counts.reset_index(inplace = True)
    return group_counts

#function to return % racial distribution in each zip code...don't think this is accurate
def subgroup_distribution(df, location, racial_majority):
    group = df.groupby([location, racial_majority]).size()
    group_pcts = group.groupby(level=0).apply(lambda x: 100 * x/float(x.sum()))
    group_pcts = pd.DataFrame(group_pcts.unstack(1))
    group_pcts.reset_index(inplace=True)
    return group_pcts

In [39]:
def map_feature_by_zipcode(zipcode_data, col):
    """
    Generates a folium map of NYC
    :param zipcode_data: zipcode dataset
    :param col: feature to display
    :return: m
    """

    # Initialize Folium Map with NYC latitude and longitude
    m = folium.Map(location=[40.81, -73.90], zoom_start=9,
                   detect_retina=True, control_scale=False)

    # Create choropleth map
    m.choropleth(
        geo_data=geo_data,
        name='choropleth',
        data=zipcode_data,
        # col: feature of interest
        columns=['zipcode', 'group_counts', 'group_pcts'],
        key_on='feature.properties.ZIPCODE',
        fill_color='OrRd',
        fill_opacity=0.9,
        line_opacity=0.2,
        legend_name='Evictions' + col
    )

    folium.LayerControl().add_to(m)

    # Save map based on feature of interest
    m.save(col + '.html')

    return m

Ended Here
==============================

### Variable:  Evictions (filings_2020)
`filings_2020` is the independent variable in this study.  The Eviction Lab data reports both filings_2020 which is a reported number per month since 2020 and filings_avg which is the average per month.  We are exploring both versions in this project.]

In [ ]:
df2.filings_2020.describe()                          

According to the preliminary descriptive statistics, evictions were on average 17.075 per zipcode, with a minimum of zero and a maximum of 550.  The interquartile range varied from 0 to 15 for the middle 50% of zipcodes.  

In order to refine the evictions into a recognizable pattern, I will divide into five categorical levels of evictions (0, 1-9, 10-29, 30-59, 60-99, and >100). This will give a more detailed attention to the extreme ranges of evictions, in order to isolate these groups from the lower rates.

In [ ]:
def evict_b(y):                                 
    '''
    INPUT: 
    y: int, from -1 to 550, the inputs of the int variable `filings_2020`
    
    OUTPUT:
    0 recoded to '<1'
    1-9 recoded to '1-9'
    10-29 recoded to '10-29'
    30-59 recoded to '30-59'
    60-99 recoded to '60-99'
    >100 recoded to '>100'
    '''
    if y == 0:
        return '0'
    if y >0 and y<10:
        return '1-9'
    elif y >= 10 and y<30:
        return '10-29'
    elif y >= 30 and y<60:
        return '30-59'
    elif y>=60 and y<100:
        return '60-99'
    elif y>=100:
        return '>100'
    else:
        return np.nan                        # missing is coded as nan 

# apply the function to `filings_2020`

df2['filings_cat'] = df2.filings_2020.apply(evict_b)

In [ ]:
# double check whether the transformation is successful:

df2[['filings_cat']]

Now that we have groups `filings_2020` into groups, let's see the resulting distribution. 

In [ ]:
with plt.style.context('fast'):
    df2.groupby('filings_cat').size().plot(kind='bar')   #bar graph in order
plt.title('Eviction Rates by Groups (Eviction Lab 2020-2022)')
plt.xlabel('Eviction by Groups')
plt.ylabel('# Evictions')

## 6. References <a id=6></a>

### Programming References:
Matplotlib Style Sheets Reference.  https://matplotlib.org/stable/gallery/style_sheets/style_sheets_reference.html

Legend in Matplotlib.  https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.legend.html

Stats t-test in Scipy.  https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html


### Datasets:
Crosswalk Dataset of Zip to Tract. U.S. Department of Housing and Urban Development. Office of Policy Development and Research.  https://www.huduser.gov/portal/datasets/usps_crosswalk.html#data

"Fair Market Rents: 40th Percentile." U.S. Department of Housing and Urban Development. Office of Policy Development and Research. Datasets.  https://www.huduser.gov/portal/datasets/fmr.html#2022_data

Peter Hepburn, Renee Louis, and Matthew Desmond. Eviction Tracking System: Version 1.0. Princeton: Princeton University, 2020. www.evictionlab.org.

### General Reference
"Summary: Fair Market Rents." U.S. Department of Housing and Urban Development. Office of Policy Development and Research. Blog.  https://www.huduser.gov/periodicals/ushmc/winter98/summary-2.html 

U.S. Commission on Civil Rights. Racial Discrimination and Eviction Policies and Enforcement in New York. 10 Mar 2022.  https://www.usccr.gov/reports/2022/racial-discrimination-and-eviction-policies-and-enforcement-new-york

Zaveri, Mihir.  After a Two-Year Dip, Evictions Accelerate in New York. The New York Times. 2 May 2022. https://www.nytimes.com/2022/05/02/nyregion/new-york-evictions-cases.html


<div class = "alert alert-info">

[Back to top](#7)<br>
    
</div>
<hr>